<a href="https://colab.research.google.com/github/nan-park/cp2_project/blob/main/AugSBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a266e9500a54ffd2c2f4a7144555687ca945fbf83a7fe88a7516b4da443bd6f1
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [11]:
import pandas as pd
import math
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import models, util
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
import torch
import tqdm


In [3]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
word_embedding_model = models.Transformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS', max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# step 1: set train data
filepath = '/content/drive/MyDrive/cp2/review_w_only_labels (3).csv'
df = pd.read_csv(filepath)
columns = df.columns[1:].tolist()
columns.remove('review_tags')
columns.remove('target')
df = df[columns]
targets = df.columns.tolist()[1:]

df.reset_index(inplace=True)
df_melt = pd.melt(df, id_vars=['index', 'review'])
df_melt[df_melt['index']<500]

,index,review,variable,value
0,0,고기도 구워주시고 친절도 하시고 맛도 최고입니다.,가성비,0
1,1,가게 인기가 좋아서 그런지 가게가 바빴는데 직원분들 저너무 친절하시구 초벌 도와주신...,가성비,0
2,2,공간도 깔끔하고 맛있습니다.,가성비,0
3,3,웨이팅은 했지만 깔끔하고 송년모임 하기 좋은것같아요 다음에 올때는 6시전에 좀 일찍...,가성비,0
4,4,직원분들이 친절하고 좋아용.리뷰만 보고 방문했는데 이렇게 분위기 좋은 고깃집은 첨이...,가성비,0
...,...,...,...,...
37091,495,사장님도 이모님들도 모두 친절하시고 음식도 맛있어요.줄서서 먹는 이유를 너무 잘 알...,혼자,0
37092,496,직원분들 엄청 친절하시고 이 가격에 말이 안되는 조합이네요 알 엄청 꽉 찼고 신선해요.,혼자,0
37093,497,친절하시고 맛있어요.,혼자,0
37094,498,게장 비싸서 잘 못 먹는데 여긴 싸고 맛있어서 배터지게 먹었습니다.너무 잘먹고갑니다,혼자,0


In [6]:
num_epochs = 1
batch_size = 16

cross_encoder_path = 'augSBERT_output/cross_encoder'
bi_encoder_path = 'augSBERT_output/bi_encoder'

df_gold = df_melt[df_melt['index']<500].reset_index(drop=True) # 7500개
df_eval = df_melt[(500<=df_melt['index']) & (df_melt['index']<700)].reset_index(drop=True)    # 3000개
gold_samples = []
dev_samples = []

for i in range(len(df_gold)):
    gold_samples.append(InputExample(texts=[df_gold.loc[i, 'review'], df_gold.loc[i, 'variable']], label=df_gold.loc[i, 'value']))
for i in range(len(df_eval)):
    dev_samples.append(InputExample(texts=[df_eval.loc[i, 'review'], df_eval.loc[i, 'variable']], label=df_eval.loc[i, 'value']))
train_dataloader = DataLoader(gold_samples, shuffle=True, batch_size=batch_size)
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='sts-dev')

# configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data

In [7]:
# step 2: train cross_encoder with STSbenchmark (gold samples)
cross_encoder.fit(train_dataloader=train_dataloader,
                  evaluator=evaluator,
                  epochs=num_epochs,
                  evaluation_steps=1000,
                  warmup_steps=warmup_steps,
                  output_path=cross_encoder_path)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/469 [00:00<?, ?it/s]

In [8]:
# step 3: Find similar(silver) sentence pairs to label
silver_data = []
sentences = set()

for sample in gold_samples:
  sentences.update(sample.texts)

sentences = list(sentences) # unique sentences
sent2idx = {sentence: idx for idx, sentence in enumerate(sentences)} # storing id and sentence in dictionary
duplicates = set((sent2idx[data.texts[0]], sent2idx[data.texts[1]]) for data in gold_samples) # not to include gold pairs of sentences again

In [9]:
# For simplicity we use a pretrained model
semantic_model_name = 'paraphrase-MiniLM-L6-v2'
semantic_search_model = SentenceTransformer(semantic_model_name)

In [14]:
# encoding all unique sentences present in the training dataset
embeddings = semantic_search_model.encode(sentences, batch_size=batch_size, convert_to_tensor=True)

progress = tqdm.tqdm(unit="docs", total=len(sent2idx))
top_k = 3
for idx in range(len(sentences)):
    sentence_embedding = embeddings[idx]
    cos_scores = util.cos_sim(sentence_embedding, embeddings)[0]
    cos_scores = cos_scores.cpu()
    progress.update(1)

    #We use torch.topk to find the highest 5 scores
    top_results = torch.topk(cos_scores, k=top_k+1)
    
    for score, iid in zip(top_results[0], top_results[1]):
        if iid != idx and (iid, idx) not in duplicates:
            silver_data.append((sentences[idx], sentences[iid]))
            duplicates.add((idx,iid))

# progress.reset()
progress.close()

100%|██████████| 515/515 [00:00<00:00, 1390.95docs/s]


In [ ]:
silver_scores = cross_encoder.predict(silver_data)